# UnPickle Sage Data
Unpickle data created on another system with Sage.  Should then be able to create a transaction trial balances.

In [1]:
import datetime as dt
import pandas as pd

from h3_yearend import p


In [2]:
import pickle
sage_2015 = pickle.load(open('sage2015.pickle', 'rb'))
df = sage_2015.df
print(len(df))
df.head(2)

56830


,TRAN_NUMBER,TYPE,DATE,ACCOUNT_REF,ALT_REF,INV_REF,DETAILS,TAX_CODE,AMOUNT,FOREIGN_AMOUNT,BANK_FLAG,DATE_BANK_RECONCILED
0,1,SI,2015-02-03,1100,X458,55774,SGL S/F COTTON TOPPER,T1,64.44,64.44,-,NaN
1,2,SI,2015-02-03,1100,X458,55774,KNG S/F COTTON TOPPER,T1,80.10,80.10,-,NaN


# Get Data for one year for one nominal account

In [3]:
# tb = sage_2015.transactional_trial_balance(dt.date(2015, 1, 1), dt.date(2016, 1, 31))
# tb.head()
accruals = df[(df['ACCOUNT_REF'] == 2109) & (df['DATE'] >= dt.date(2015, 2, 1)) & (df['DATE'] <= dt.date(2016, 1, 31))]
accruals.head()

,TRAN_NUMBER,TYPE,DATE,ACCOUNT_REF,ALT_REF,INV_REF,DETAILS,TAX_CODE,AMOUNT,FOREIGN_AMOUNT,BANK_FLAG,DATE_BANK_RECONCILED
3252,1482,PI,2015-02-19,2109,BTP001,A05,Phone,T1,1147.73,1147.73,-,NaN
3255,1483,PI,2015-02-11,2109,BTP001,A06,Phone,T9,158.31,158.31,-,NaN
3288,1494,PI,2015-02-10,2109,EON001,A17,Electricity Bill,T5,7.26,7.26,-,NaN
3291,1495,PI,2015-02-10,2109,EON001,A18,Electricity Bill,T5,9.35,9.35,-,NaN
3294,1496,PC,2015-02-11,2109,EON001,A18,Electricity Bill on reading,T5,-0.20,-0.20,-,NaN


## Temporarily save data to spreadsheet so that the data can be eyeballed.
Accruals (2109) is creditors account which is a liability so should negative.

In [4]:
print(p(accruals['AMOUNT'].sum()))

-3051.36


In [10]:
def save_to_excel(df, filename):
    temp = df.copy()
    temp['DATE']=[str(d)[0:10] for d in temp['DATE']]
    temp['DATE_BANK_RECONCILED']=[str(d)[0:10] for d in temp['DATE_BANK_RECONCILED']]
    with pd.ExcelWriter(filename) as writer:
        temp.to_excel(writer,'Accruals 2109')
save_to_excel(df, ('Accruals 2109 for checking.xlsx'))

## Summary need to Categorise
There are 260 entries - need to categorise them and summarise them.  Then deal with each group.  Start with Monthly 
accruals.

In [11]:
def categorise(row):
    result = 'Unknown'
    if row['DETAILS'] == 'Monthly Accruals':
        result = 'Monthly Accruals'
    return result

accruals['Category'] = accruals.apply(categorise, axis=1)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
pd.pivot_table(accruals,index=["Category"], values=['TYPE'], aggfunc='count')

,TYPE
Category,
Monthly Accruals,12
Unknown,248


In [49]:
accruals_list = list(accruals[accruals['Category']=='Monthly Accruals']['TRAN_NUMBER'])
assert len(accruals_list) == 12
accruals_list

[5991, 6005, 6019, 8784, 8797, 12265, 12278, 15521, 15534, 16455, 18815, 21534]

### Look around and accruals and see how to get whole transaction
Is there an explicit link or going to need to seom type of inference

In [14]:
df[(df['TRAN_NUMBER'] >= 5991-5) & (df['TRAN_NUMBER'] <= 5991+5)]

,TRAN_NUMBER,TYPE,DATE,ACCOUNT_REF,ALT_REF,INV_REF,DETAILS,TAX_CODE,AMOUNT,FOREIGN_AMOUNT,BANK_FLAG,DATE_BANK_RECONCILED
14890,5986,JD,2015-02-28,8402,8402,Accruals Draft,Admin Insurance,T9,170,170,-,NaN
14891,5987,JD,2015-02-28,8405,8405,Accruals Draft,Admin Motor,T9,365,365,-,NaN
14892,5988,JD,2015-02-28,8410,8410,Accruals Draft,Admin Rent & Rates,T9,2020,2020,-,NaN
14893,5989,JD,2015-02-28,8426,8426,Accruals Draft,Admin Telephones,T9,700,700,-,NaN
14894,5990,JD,2015-02-28,8430,8430,Accruals Draft,Admin IT/EDI,T9,130,130,-,NaN
14895,5991,JC,2015-02-28,2109,2109,Accruals Draft,Monthly Accruals,T9,-13400,-13400,-,NaN
14896,5992,JD,2015-03-31,7100,7100,Accruals Draft Mar15,Works Gas & Elec,T9,750,750,-,NaN
14897,5993,JD,2015-03-31,7105,7105,Accruals Draft Mar15,Works Sundries,T9,135,135,-,NaN
14898,5994,JD,2015-03-31,7200,7200,Accruals Draft Mar15,Works Depreciation,T9,35,35,-,NaN
14899,5995,JD,2015-03-31,7202,7202,Accruals Draft Mar15,Works Insurance,T9,920,920,-,NaN


## Given a transaction number find the journal entries
- Find transaction
- go forwards until not JC,JD and same date
- go backwards until not JC,JD and same date (collecting stop if no long the same or if all adds up to zero)
- if not including target restart going backwards until target included.
- Make sure all adds up to Zero

In [47]:
class TransactionError(Exception):
    pass

def get_entry(df, number, direction=1):  # Default direction is searching forwards
    transactions = df[(df['TRAN_NUMBER'] == number)]
    if len(transactions) > 0:  #  there may be more eg PI consists of three transactions.
        return transactions.iloc[0]
    else:  # probably item has been deleted so try next one
        # TODO make sure code works at beginging and end of data.  Currently raises a 
        #  transaction error
        min = df['TRAN_NUMBER'].min()
        max = df['TRAN_NUMBER'].max()
        if number < min:
            raise TransactionError('Searching for transaction and are looking for ' +
                                  'a transaction less than the minimum. {}'.format(number))
        elif number > max:
            raise TransactionError('Searching for transaction and are looking for ' +
                                  'a transaction greater than the maximum. {}'.format(number))
        else:
            return get_entry(df, number+direction, direction)  # Recursivly get next

class Transaction:
    
    def __init__(self):
        self.transaction = {}
    
    def sum(self):
        result = p(0)
        for k,v in iter(self.transaction.items()):
            result += v
        return result

    def append(self, nominal_code, value):
        if nominal_code in self.transaction:
            raise TransactionError('Attempting to add two entries to the same nominal code {} into {}'.\
                                   format(nominal_code, self.transaction))
        else:
            self.transaction[nominal_code] = value
        
    def is_valid(self):
        return (len(self.transaction)  > 0) and (self.sum() == p(0))


def return_transaction(df, one_entry):
    
    def journal_matches(test, date):
        return (test['DATE'] == date) and (test['TYPE'] in ('JC', 'JD'))

    def search_backwards(one_entry, start, date):
        #Search backwards adding transactions until complete and one_entry in transactions
        index = start
        transaction = Transaction()
        test = get_entry(df, index, direction=-1)
        while journal_matches(test, date):
            if test['TYPE'] == 'JD':
                sign=1
            else:
                sign=-1
            transaction.append(test['ACCOUNT_REF'],  p(test['AMOUNT']))  # p(sign) *
            if transaction.is_valid():
                if index <= one_entry:  # Found the transaction
                    return transaction
                else:  # there are probably two or more transactions on the same day
                    return search_backward(one_entry, index, date)
            index -= 1  # Else keep on trying to add transactions until it is valid
            test = get_entry(df, index, direction=-1)
        raise TransactionError("""The transaction could not be found or 
doesn't add up to zero.{} {}. Start = {}""".format(
                transaction.sum(), transaction.transaction, start))
        
    index = one_entry
    test = get_entry(df, index)
    date = test['DATE']
    test = get_entry(df, index+1)
    while journal_matches(test, date):
        test = get_entry(df, index)
        index += 1
    high_water_mark = index
    print('End of transactions is {} on {}'.format(index, date))
    return search_backwards(one_entry, high_water_mark, date)
    
a = return_transaction(df, 5991)
a.transaction

End of transactions is 5991 on 2015-02-28 00:00:00


{2109: Decimal('-13400.00'),
 7100: Decimal('750.00'),
 7105: Decimal('135.00'),
 7200: Decimal('35.00'),
 7202: Decimal('920.00'),
 7204: Decimal('7500.00'),
 8200: Decimal('300.00'),
 8300: Decimal('250.00'),
 8400: Decimal('125.00'),
 8402: Decimal('170.00'),
 8405: Decimal('365.00'),
 8410: Decimal('2020.00'),
 8426: Decimal('700.00'),
 8430: Decimal('130.00')}

In [48]:
[return_transaction(df,x) for x in accruals_list]

End of transactions is 5991 on 2015-02-28 00:00:00
End of transactions is 6005 on 2015-03-31 00:00:00
End of transactions is 6019 on 2015-04-30 00:00:00
End of transactions is 8784 on 2015-05-31 00:00:00
End of transactions is 8797 on 2015-06-30 00:00:00
End of transactions is 12265 on 2015-07-31 00:00:00
End of transactions is 12278 on 2015-08-31 00:00:00
End of transactions is 15521 on 2015-09-30 00:00:00
End of transactions is 15534 on 2015-10-31 00:00:00
End of transactions is 16461 on 2015-11-30 00:00:00


TransactionError: The transaction could not be found or 
doesn't add up to zero.0.00 {}. Start = 16461

In [43]:
 df['TRAN_NUMBER'].max()

22648